# Expectation-Maximization Clustering

### Maximization
$$
q_{mk} = \frac{\sum\limits_{n=1}^{N} r_{nk}I(t_m \in d_n)}{\sum\limits_{n=1}^{N}r_{nk}};
\alpha_{k} = \frac{1}{N} \sum\limits_{n=1}^{N} r_{nk}
$$

### Expectation
$$
r_{nk} = \frac{\alpha_k\left(\prod_{t_m \in d_n}q_{mk}\right)
\left(\prod_{t_m \not\in d_n}(1-q_{mk})\right)}
{\sum\limits_{k=1}^{K}\alpha_k\left(\prod_{t_m \in d_n}q_{mk}\right)
\left(\prod_{t_m \not\in d_n}(1-q_{mk})\right)}
$$

### Example
<table>
    <tr>
        <th>DocID</th>
        <th>Tokens</th>
        <th>Class</th>
    </tr>
    <tr>
        <td>0</td>
        <td>apple ios mac book fruit</td>
        <td>A</td>
    </tr>
    <tr>
        <td>1</td>
        <td>apple mac book apple store fruit</td>
        <td>A</td>
    </tr>
    <tr>
        <td>2</td>
        <td>microsoft ibm apple oracle</td>
        <td>A</td>
    </tr>
    <tr>
        <td>3</td>
        <td>apple banana mango fruit</td>
        <td>B</td>
    </tr>
    <tr>
        <td>4</td>
        <td>apple fruit</td>
        <td>B</td>
    </tr>
</table>

In [1]:
import numpy as np
from IPython.core.display import display, HTML, Image

docs = [
    ['apple', 'ios', 'mac', 'book', 'fruit'],
    ['apple', 'mac', 'book', 'apple', 'store'],
    ['microsoft', 'ibm', 'apple', 'oracle'],
    ['apple', 'banana', 'mango', 'fruit'],
    ['apple', 'fruit', 'mango']
]
terms = list(set([x for y in docs for x in y]))
M = np.array([[1 if x in y else 0 for x in terms] for y in docs])
N, K, m = len(docs), 2, len(terms)
A = np.zeros(K)
Q = np.zeros((m, K))

## Random init $r_{nk}$

In [2]:
R = np.zeros((N, K))
for doc in range(N):
    a = np.random.uniform()
    R[doc] = [a, 1-a]

In [3]:
R

array([[0.05605402, 0.94394598],
       [0.44017889, 0.55982111],
       [0.44729537, 0.55270463],
       [0.0036333 , 0.9963667 ],
       [0.68571744, 0.31428256]])

## Functions

In [4]:
def maximization(K, R):
    for k in range(K):
        A[k] = R[:,k].sum() / N
        for word in range(m):
            sigma_doc = 0.0
            for doc in range(N):
                sigma_doc += R[doc,k] * M[doc,word]
            Q[word][k] = sigma_doc / R[:,k].sum()

In [5]:
def estimate(doc, k, Q, A):
    q_doc = np.zeros(m)
    for word in range(m):
        if M[doc,word] > 0:
            q_doc[word] = Q[word,k]
        else:
            q_doc[word] = 1 - Q[word,k]
    return A[k] * q_doc.prod()
    
def expectation(K, Q, A):
    for doc in range(N):
        k_estimation = np.array([estimate(doc, k, Q, A) for k in range(K)])
        for k in range(K):
            R[doc][k] = k_estimation[k] / k_estimation.sum()

In [8]:
def to_table(title, data, cols, rows):
    header = "<tr>" + "".join(["<th>{}</th>".format(x) for x in [''] + list(cols)]) + "</tr>"
    trs = []
    for i, c in enumerate(rows):
        tr = "<tr>" + "<td>{}</td>".format(c)
        tr += "".join(["<td>{}</td>".format(round(x, 3)) for x in data[i]])
        tr += "</tr>"
        trs.append(tr)
    table = "<h3>{}</h3><table>{}{}</table>".format(
        title,
        header,
        "".join(trs)
    )
    return table

def show(r, q, a):
    table = "<table><tr><td style='vertical-align: top;'>{}</td><td style='vertical-align: top;'>{}</td><td style='vertical-align: top;'>{}</td></tr></table>".format(
        r, q, a
    )
    display(HTML(table))

## Start

In [9]:
TR = to_table('$r_{nk}$', R, range(K), range(N))
TQ = to_table('$q_{mk}$', Q, range(K), terms)
TA = to_table('$a_{k}$', [A], range(K), ['priors'])
show(TR, TQ, TA)

,0,1
0,0.056,0.944
1,0.44,0.56
2,0.447,0.553
3,0.004,0.996
4,0.686,0.314
,0,1
mac,0.0,0.0
ibm,0.0,0.0
book,0.0,0.0
banana,0.0,0.0


## Iterate

In [10]:
for iteration in range(10):
    maximization(K, R)
    expectation(K, Q, A)
    TR = to_table('$r_{nk}$', R, range(K), range(N))
    TQ = to_table('$q_{mk}$', Q, range(K), terms)
    TA = to_table('$a_{k}$', [A], range(K), ['priors'])
    display(HTML("<h2>ITERATION {}</h2>".format(iteration+1)))
    show(TR, TQ, TA)

,0,1
0,0.014,0.986
1,0.414,0.586
2,0.902,0.098
3,0.003,0.997
4,0.383,0.617
,0,1
mac,0.304,0.447
ibm,0.274,0.164
book,0.304,0.447
banana,0.002,0.296


,0,1
0,0.0,1.0
1,0.203,0.797
2,1.0,0.0
3,0.0,1.0
4,0.031,0.969
,0,1
mac,0.249,0.479
ibm,0.526,0.03
book,0.249,0.479
banana,0.002,0.304


,0,1
0,0.0,1.0
1,0.004,0.996
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
,0,1
mac,0.165,0.477
ibm,0.81,0.0
book,0.165,0.477
banana,0.0,0.266


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
,0,1
mac,0.004,0.5
ibm,0.996,0.0
book,0.004,0.5
banana,0.0,0.25


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
,0,1
mac,0.0,0.5
ibm,1.0,0.0
book,0.0,0.5
banana,0.0,0.25


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
,0,1
mac,0.0,0.5
ibm,1.0,0.0
book,0.0,0.5
banana,0.0,0.25


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
,0,1
mac,0.0,0.5
ibm,1.0,0.0
book,0.0,0.5
banana,0.0,0.25


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
,0,1
mac,0.0,0.5
ibm,1.0,0.0
book,0.0,0.5
banana,0.0,0.25


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
,0,1
mac,0.0,0.5
ibm,1.0,0.0
book,0.0,0.5
banana,0.0,0.25


,0,1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
,0,1
mac,0.0,0.5
ibm,1.0,0.0
book,0.0,0.5
banana,0.0,0.25
